In [416]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import hvplot.pandas
from pathlib import Path  
import panel as pn
pn.extension('plotly')

# make plots static
%matplotlib inline

In [417]:
url = "https://www.catch.com.au/shop/new-to-catch/event/footwear-150983/?st=1"


In [418]:
# Use the below headers when requesting the url may help you to avoid being blocked from the site
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

#result = requests.get(url, hea)

#doc = BeautifulSoup(result.text,"html.parser")
#credits to Khuong

In [419]:
# Get the HTML content get_url()
# credits to Khuong Tran
def get_url(url):
    try:
        response = requests.get(url,headers = HEADERS).text
        soup = BeautifulSoup(response, 'html.parser')
        return soup
    except Exception as err:
        print('ERROR BY REQUEST:', err)
        return None
    

In [420]:
get_url(url).prettify()[0:1000]

'<!DOCTYPE html>\n<html class="no-js no-webp" lang="en">\n <head>\n  <link as="font" crossorigin="" href="https://s.catch.com.au/static/catch/fonts/biennale_regular.woff2" rel="preload" type="font/woff2"/>\n  <link as="font" crossorigin="" href="https://s.catch.com.au/static/catch/fonts/biennale_semibold.woff2" rel="preload" type="font/woff2"/>\n  <link as="font" crossorigin="" href="https://s.catch.com.au/static/catch/fonts/biennale_black.woff2" rel="preload" type="font/woff2"/>\n  <script>\n   var cotd = cotd || {};\n  </script>\n  <script crossorigin="anonymous" src="https://polyfill.io/v3/polyfill.min.js?features=es6%2CNodeList.prototype.%40%40iterator%2CPromise%2CPromise.prototype.finally%2CglobalThis%2CObject.values%2CObject.entries%2CObject.assign%2ClocalStorage%2CArray.prototype.includes%2CArray.prototype.flatMap%2CArray.prototype.find%2CString.prototype.includes%2CString.prototype.startsWith%2CSymbol.toStringTag%2CNumber.isNaN%2Cfetch%2CElement.prototype.prepend%2Csmoothscroll

In [421]:
soup = get_url(url)


In [422]:
#Obtaining main data body
data_body = soup.find_all("div" , {"class" : "product"})

#Check number of items 
print(len(data_body))

84


In [423]:
#extract item pricing - "TESTING"
price = data_body[2].find("div" , {"class": "price--container onepass"}).get_text()

#.strip() to remove empty spaces
print(price)
print(price.strip())


                                                  $119.99  
$119.99


In [424]:
#test run for items on Sale "Don't pay XXX Save XXX etc"
testing = data_body[1].find("div" , {"class": "price--main theme"}).get_text()

#convert text to float
print(testing)
testing_2 = float(testing.replace('$', ''))
print(testing_2)

$119.99
119.99


In [425]:
#create empty dataframes to be appended on

product_name = []
product_price = []
product_savings = []
product_brands = []

for item in range(0,len(data_body)):

    #Extract Product Names
    try:
        name = data_body[item].find("a" , {"class": "js-product-link product--title-link js-dnt"}).get_text()
        product_name.append(name)  
    
    except:
        product_name.append('Empty')   
    
    #Extract Product's Brand
    try:
        brands = data_body[item].find("div" , {"class": "product--brand"}).get_text()
        product_brands.append(brands)  
    
    except:
        product_brands.append('Not Available')
    
    #Extract Product's Price
    #delete "$" and convert price to float (just incase we need to sort them)
    try:
        price = data_body[item].find("div" , {"class": "price--container onepass"}).get_text().strip()
        price = float(price.replace('$', ''))
        product_price.append(price)
    
    #Code will run into an error if an item is on sale because price will be assigned to "price--main theme"
    #This code will extract price data from price--main theme instead
    except:
        price2 = data_body[item].find("div" , {"class": "price--main theme"}).get_text()
        price2 = float(price2.replace('$', ''))
        product_price.append(price2)
     
    #Extract Product's Savings   
    try:
        savings = data_body[item].find("span" , {"data-ts": "savings-value"}).get_text()
        savings = float(savings.replace('$', ''))
        product_savings.append(savings)
        
    except:
        product_savings.append('-')      

d = {'Product Name': product_name, 'Brand': product_brands, 'Price ($AUD)': product_price, "Savings!": product_savings}

cotd_df = pd.DataFrame(data=d)
cotd_df

,Product Name,Brand,Price ($AUD),Savings!
0,Birkenstock Unisex Mayari Birko-Flor Regular F...,Birkenstock,119.99,36.0
1,Birkenstock Unisex Mayari Regular Fit Sandals ...,Birkenstock,119.99,24.0
2,Birkenstock Arizona Leather Regular Fit Sandal...,Birkenstock,119.99,-
3,Birkenstock Unisex Arizona Narrow Sandals - Ha...,Birkenstock,119.99,80.0
4,OZWEAR Connection Unisex Button Ugg Boots - Ch...,Ozwear Connection,99.99,139.0
...,...,...,...,...
79,Jessica Simpson Women's Plush Crossover Slides...,Jessica Simpson,19.99,-
80,AT 55332Z Lace Up Zip Side Boot - Wheat,Oliver,189.50,-
81,Verali Women's Luella Heeled Mules - Black,Verali,79.95,10.0
82,Verali Women's Luella Heeled Mules - Taupe,Verali,79.95,10.0


In [426]:
most_exp_price = max(cotd_df["Price ($AUD)"])
cheapest_price = min(cotd_df["Price ($AUD)"])

most_exp_item = cotd_df[cotd_df["Price ($AUD)"] == most_exp_price]['Product Name'].values[0]
cheapest_item = cotd_df[cotd_df["Price ($AUD)"] == cheapest_price]['Product Name'].values[0]

print(f"Most expensive item: {most_exp_item}")
print(f"Item Price: ${most_exp_price}")
print(f"Cheapest item: {cheapest_item}")
print(f"Item Price: ${cheapest_price}")

Most expensive item: Timberland Women's Heritage 6" Waterproof Boots - Medium Grey/Black/Floral
Item Price: $199.99
Cheapest item: Snuggups Women's Quote Coffee & Then The World Slippers - Pink
Item Price: $17.99


In [427]:
data = {' ': 
        ["Most Expensive Item","Item Price","Cheapest Item","Item Price"],
        "": 
        [most_exp_item, most_exp_price, cheapest_item, cheapest_price]}

price_summary_df = pd.DataFrame(data=data).set_index(" ")
price_summary_df

,
,
Most Expensive Item,"Timberland Women's Heritage 6"" Waterproof Boot..."
Item Price,199.99
Cheapest Item,Snuggups Women's Quote Coffee & Then The World...
Item Price,17.99


In [428]:
#extracting products with values in "Savings!" column
savings_df = cotd_df.loc[cotd_df['Savings!'] != '-']
savings_df

,Product Name,Brand,Price ($AUD),Savings!
0,Birkenstock Unisex Mayari Birko-Flor Regular F...,Birkenstock,119.99,36.0
1,Birkenstock Unisex Mayari Regular Fit Sandals ...,Birkenstock,119.99,24.0
3,Birkenstock Unisex Arizona Narrow Sandals - Ha...,Birkenstock,119.99,80.0
4,OZWEAR Connection Unisex Button Ugg Boots - Ch...,Ozwear Connection,99.99,139.0
6,OzWild Unisex Classic Short Ugg Boots - Chestnut,Ozwear Connection,79.99,19.0
7,OzWild Unisex Classic Short Ugg Boots - Black,Ozwear Connection,79.99,19.0
9,OZWEAR Connection Women's Button Ugg Boot - Ch...,Ozwear Connection,99.99,139.0
10,OzWild Unisex Classic Short Ugg Boots - Grey,Ozwear Connection,79.99,19.0
11,OZWEAR Connection Ugg Women's Genesis Slippers...,Ozwear Connection,49.99,49.0
12,OZWEAR Connection Unisex Classic Mini Ugg Boot...,Ozwear Connection,85.99,113.0


In [429]:
#pivoting of "Brand" by their mean values
price_df = savings_df.groupby("Brand").mean()
price_df = price_df.rename(columns={'Price ($AUD)': 'Mean Savings'})

#Item count of "Brands" which is on sale
brand_count = savings_df['Brand'].value_counts(dropna=False)
brand_df = pd.DataFrame(brand_count)
brand_df.rename(columns={'Brand':'No. of items on Sale'}, inplace=True)

#Merging data based on Index
summary_df = pd.merge(price_df, brand_df, left_index=True, right_index=True)

#Sort by Mean Savings
summary_df.sort_values(by=["No. of items on Sale"],ascending=False)

summary_df.hvplot.table(columns=['index', 'Mean Savings', 'No. of items on Sale'], sortable=True, selectable=True)

:Table   [index,Mean Savings,No. of items on Sale]

In [430]:
items_brand_plot = summary_df.hvplot.bar(title="No. of Sales Item", y='No. of items on Sale',rot=90,height=500)
items_brand_plot

:Bars   [index]   (No. of items on Sale)

In [431]:
items_savings = summary_df.hvplot.bar(title="Mean Savings of Brands", y='Mean Savings',rot=90,height=500)
items_savings

:Bars   [index]   (Mean Savings)

In [ ]:
#export to csv
filepath = Path('..\Web-Scraping-Project\catch_of_the_day.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
cotd_df.to_csv(filepath)